In [ ]:
import sys
sys.path.append('../utils/')

In [ ]:
from ImageUtils import *

In [ ]:
import numpy as np
import pandas as pd # Needs the package Pandas to be installed. Check Anaconda Environments and Packages.
from sklearn.decomposition import PCA # Needs SciKit Learn package to be installed. Check Anaconda Environments and Packages.4
from sklearn.covariance import LedoitWolf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score
import matplotlib.pyplot as plt
from scipy.spatial.distance import mahalanobis
from collections import Counter

# DATASET FACES 94

## Data preprocessing

In order to prepare the image dataset to proceed with the final stages of modeling and valuation processes. It is unavoidable to standardize the images features on the initial dataset (codename - faces94) and the external images use it on the future testing activities under the following conditions:

**General Images Characteristics**:

* File Format *.jpg
* Images on Gray Scale.
* Size 180x200 for the images on the dataset (codename- faces94)

**Activities**:

* Organize the images on a short-listed to prepare a new dataset.
* Exclude from the dataset all the images without the *.jpg format.
* On the OpenCV library of Python, upload the images and storage the matrix in arrays for numeric treatment.
* On the OpenCV library of Python, change the images to gray-scale and resize the photos to 180x200.
* Finally, the outcome is a new dataset with the proper images for testing and modeling face recognition. With the Eigenfaces model to apply the Principal Component Analysis (PCA) so represent the face images in a low dimension.


In [ ]:
face94_male = readFaces94MaleFaces(gray=True)
plt.imshow(face94_male[0], plt.cm.gray);

In [ ]:
N, height, width = face94_male.shape

In [ ]:
labels_faces = np.ones(N)

# Mean face

In [ ]:
mean_face = np.mean(face94_male.reshape(N, height*width), axis=0).reshape(height, width)
plt.imshow(mean_face, cmap=plt.cm.gray);

# Median face

In [ ]:
median_face = np.median(face94_male.reshape(N, height*width), axis=0).reshape(height, width)
plt.imshow(median_face, cmap=plt.cm.gray);

# Images of natual landscapes

The landscape images were obtain of **ImageNet** database [ImageNet database](http://image-net.org/) , 
each one of the directions is [online](http://image-net.org/api/text/imagenet.synset.geturls?wnid=n13104059). We use cv2 package by read and resize images, then we create an Numpy array with a gray scale of images.

In [ ]:
landscapes = np.array(readLandsCapeImage(gray=True)) # Read dataset
plt.imshow(landscapes[45], plt.cm.gray); # show image

In [ ]:
labels_landscapes = np.zeros(landscapes.shape[0])

In [ ]:
dataset = np.vstack((face94_male, landscapes))
plt.imshow(dataset[-1], plt.cm.gray);

In [ ]:
labels = np.concatenate((labels_faces, labels_landscapes))

In [ ]:
dataset_N, height, width = dataset.shape

In [ ]:
mean_with_noise = np.mean(dataset.reshape(dataset_N, height*width), axis=0).reshape(height, width)
plt.imshow(mean_with_noise, cmap=plt.cm.gray);

In [ ]:
fig = plt.figure(figsize=(8,10))
ax1 = fig.add_subplot(1,2,1)
plt.title("mean with noise")
ax1.imshow(mean_with_noise, plt.cm.gray)
ax2 = fig.add_subplot(1,2,2)
plt.title("mean")
ax2.imshow(mean_face, plt.cm.gray)
Dis=np.linalg.norm(mean_with_noise - mean_face, ord=2, keepdims=False)
print("Distance "+ str(Dis))

In [ ]:
median_with_noise = np.median(dataset.reshape(dataset_N, height*width), axis=0).reshape(height, width)
plt.imshow(median_with_noise, cmap=plt.cm.gray);

In [ ]:
fig = plt.figure(figsize=(8,10))
ax1 = fig.add_subplot(1,2,1)
plt.title("median with noise")
ax1.imshow(median_with_noise, plt.cm.gray)
ax2 = fig.add_subplot(1,2,2)
plt.title("median")
ax2.imshow(median_face, plt.cm.gray)
Dis=np.linalg.norm(median_with_noise - median_face, ord=2, keepdims=False)
print("Distance "+ str(Dis))

# Median face as a real image

In [ ]:
A=dataset.reshape(dataset_N, height*width)

In [ ]:
dist_1=[]
for i in range(A.shape[0]):
    D=abs(np.subtract(A[i], A))  
    s=sum(np.sum(D,axis=0)) 
    dist_1.append(s) 
    
Min_1=np.argmin(np.array(dist_1))

plt.imshow(dataset[Min_1],plt.cm.gray)

## Show atypical data distances

In [ ]:
distance_info = getNormsAndDistanceInfoFromBaseImage(base_image=mean_with_noise, array_images=dataset, labels=labels)
visualizeOutlierInfo(distance_info,labels)

In [ ]:
print(distance_info['falsitude_metrics'])

# Show atypical data distances (outliers interquartile range)

In [ ]:
visualizeOutlierInfo2(distance_info,dataset,labels)

In [ ]:
print(distance_info['falsitude_metrics_iqr'])

# Normas: Norm1.0  Norm2.0  Norm3.0  Norminf  Norm2.5  Norm0.71

In [ ]:
selected_norm = "Norminf"
cols = 6
rows = int(np.ceil(distance_info["outliers"][selected_norm]["indices"].shape[0]/cols))
plt.figure(figsize=(180,200))
for i in np.arange(distance_info["outliers"][selected_norm]["indices"].shape[0]):
    plt.subplot(rows, cols, i + 1)
    plt.imshow(dataset[distance_info["outliers"][selected_norm]["indices"][i]], plt.cm.gray)

In [ ]:
selected_norm = "Norm1.0"
selected_outliers = "outliersiqr"
Distance=distance_info["norms"][selected_norm][distance_info[selected_outliers][selected_norm]['indices']]
Ind=distance_info[selected_outliers][selected_norm]['indices']
Distance, Ind =zip(*sorted(zip(Distance, Ind)))
cols = 6
rows = int(np.ceil(len(Ind)/cols))
plt.figure(figsize=(180,200))
for i in np.arange(len(Ind)):
    plt.subplot(rows, cols, i+1)
    plt.imshow(dataset[Ind[-(i+1)]], plt.cm.gray)

In [ ]:
faces94_male = readFaces94MaleFaces(gray=True)
faces94_female = readFaces94FemaleFaces(gray=True)
faces94_malestaff = readFaces94MaleStaffFaces(gray=True)
landscapes = np.array(readLandsCapeImage(gray=True))

dataset = np.vstack((faces94_male, faces94_female, faces94_malestaff, landscapes))

labels = np.concatenate((
    np.ones(faces94_male.shape[0]),
    np.full(faces94_female.shape[0], 2),
    np.full(faces94_malestaff.shape[0], 3),
    np.zeros(landscapes.shape[0])
))

dataset_N, height, width = dataset.shape

In [ ]:
mean_all = np.mean(dataset.reshape(dataset_N, height*width), axis=0).reshape(height, width)
plt.imshow(mean_all, plt.cm.gray)

# Principal component analysis (PCA)

In [ ]:
dataset_norm = dataset/255
dataset_norm_cov = np.cov(dataset_norm.reshape(dataset_N, height*width))

np.linalg.det(dataset_norm_cov)

In [ ]:
u,s,vh = np.linalg.svd(dataset_norm_cov)

In [ ]:
s

In [ ]:
#mean_norm = np.mean(dataset_norm.reshape(dataset_N, height*width), axis=0).reshape(height, width)

#lw = LedoitWolf(store_precision=True, assume_centered=True)
#cov_all = lw.fit(
#    np.subtract(dataset_norm, mean_norm).reshape(dataset_N, height*width)
#)

In [ ]:
representation_percentage = 0.85

In [ ]:
sum_eig = np.sum(s)
percentage_variance = np.divide(s, sum_eig)
sum_var = 0
num_var = 0
for i in np.arange(percentage_variance.shape[0]):
    if sum_var >= representation_percentage:
        num_var = i
        break;
    
    sum_var += percentage_variance[i]
    
num_var

In [ ]:
pca = PCA(n_components=num_var, whiten=True).fit(dataset.reshape(dataset_N, height*width))
pca.components_.shape;

In [ ]:
cols = 6
rows = int(np.floor(num_var/cols))
plt.figure(figsize=(180,200))
for i in np.arange(rows * cols):
    plt.subplot(rows, cols, i + 1)
    plt.imshow(pca.components_[i].reshape(height, width), plt.cm.gray)

In [ ]:
mean_face = pca.mean_.reshape(height, width)
mean_face2 = np.mean(dataset.reshape(dataset_N, height*width), axis=0).reshape(height, width)
fig = plt.figure(figsize=(8,10))
ax1 = fig.add_subplot(1,2,1)
plt.title("PCA mean")
ax1.imshow(mean_face, plt.cm.gray)
ax2 = fig.add_subplot(1,2,2)
plt.title("np mean")
ax2.imshow(mean_face2, plt.cm.gray)
Dis=np.linalg.norm(mean_face - mean_face2, ord=2, keepdims=False)
print("Distance "+ str(Dis))

In [ ]:
dataset_projected = pca.transform(dataset.reshape(dataset_N, height*width))

In [ ]:
dataset_projected.shape

In [ ]:
example_image = np.matmul(dataset_projected[1235], pca.components_)
plt.imshow(example_image.reshape(height, width), plt.cm.gray)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset_projected, labels, test_size=0.3, stratify=labels)

In [ ]:
classifier = LogisticRegression(solver='newton-cg', multi_class='multinomial')
classifier.fit(X_train, y_train)

In [ ]:
y_train_pred = classifier.predict(X_train)
y_test_pred = classifier.predict(X_test)

In [ ]:
accuracy_score(y_true=y_test, y_pred=y_test_pred)

In [ ]:
precision_score(y_true=y_test, y_pred=y_test_pred, average=None)

# Check parametric distribution of distances


It is evaluated if the metrics can be adjusted with a parametric distribution, for this purpose the chi-squared statistic is calculated (small p values indicates a better fit) and the Kolmogorov-Smirnov test (values greater than .05 indicate good fit)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
dist=distance_info['norms']['Norm1.0']
check_parametricDistribu_distances(dist)